<a href="https://colab.research.google.com/github/dustinlaa/beats-to-blocks/blob/dev/beats_to_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup Drive Mount
from google.colab import drive
drive.mount('/content/drive')
!cd /content/drive/MyDrive/beats-to-blocks

In [ ]:
#@title Upload Audio (only if file has not been previously uploaded)
from google.colab import files
import os

input_file = files.upload()
input_file_name = list(input_file.keys())[0]
file_name, file_extension = os.path.splitext(list(input_file.keys())[0])
input_file = input_file_name.split(".")[0]
output_folder = '/content/drive/MyDrive/beats-to-blocks/' + file_name + "_files"
if not os.path.exists(output_folder):
  os.mkdir(output_folder)

In [ ]:
#@title Choose output folder (if file is already uploaded) ex. fileName.wav
from google.colab import files
import os

input_file = input("File name: ")
input_file_name = os.path.splitext(input_file)[0]
output_folder = '/content/drive/MyDrive/beats-to-blocks/' + input_file_name + "_files"
input_file = input_file_name
print(output_folder)

In [ ]:
#@title Setup for Virtual Environment for Spleeter
# Virtual Environments are needed due to package incompatability
!pip install virtualenv

# Creates the virtual enviroment for splitting
!virtualenv split
# Switch to the split virtual environment
!source split/bin/activate; pip install split
!source split/bin/activate; pip install spleeter
with open('spleeter_splitter.py', 'w') as spleeter_file:
  string = '''from spleeter.separator import Separator
import sys

separator = Separator('spleeter:5stems')

input_file = sys.argv[1]
output_folder = sys.argv[2]

separator.separate_to_file(input_file, output_folder)'''
  spleeter_file.write(string)
  spleeter_file.close()

# !pip list
# !source split/bin/activate; pip list

In [ ]:
#@title Setup for Demucs
!pip install demucs
import os
import demucs
import subprocess

In [ ]:
#@title Spleeter Splitting

spleeter_output = output_folder + "/spleeter"

dir = '/content/midi_outputs/'

!source split/bin/activate; split/bin/python spleeter_splitter.py $input_file_name $spleeter_output

In [ ]:
#@title Demucs Splitting
command = ["python", "-m", "demucs.separate", input_file_name, "--out", output_folder]
subprocess.run(command, check=True)

In [ ]:
#@title Sending Demucs other.wav into Spleeter (to separate piano)
demucs_into_spleeter = output_folder + "/demucs_into_spleeter"
if not os.path.exists(demucs_into_spleeter):
  os.mkdir(demucs_into_spleeter)

demucs_input = output_folder + "/htdemucs/" + input_file + "/other.wav"

!source split/bin/activate; split/bin/python spleeter_splitter.py $demucs_input $demucs_into_spleeter

In [ ]:
#@title Install Spotify Basic Pitch
!source split/bin/activate; pip install basic-pitch
string = '''from basic_pitch.inference import predict
from basic_pitch import ICASSP_2022_MODEL_PATH
from enum import Enum
import basic_pitch.inference
import sys
import os

input_file = sys.argv[1]
output_folder = sys.argv[2]

class InstrumentIndex(Enum):
    ACOUSTIC_BASS = 32
    SYNTH_DRUM = 118
    ACOUSTIC_GRAND_PIANO = 0

song_name = input_file.split('.')[0]

# demucs

base = output_folder + '/'



# base = '/content/mafia/mafia/'
baseDemucs = base + 'htdemucs/' + song_name + '/'
os.mkdir(base + 'demucs_basicPitch')
baseDemucs_out = base + 'demucs_basicPitch/'

input_paths = [
    baseDemucs+'bass.wav',
    baseDemucs+'drums.wav',
    baseDemucs+'other.wav',
    # baseDemucs+'piano.wav',
    baseDemucs+'vocals.wav'
]

output_paths = [
    baseDemucs_out+'bass.mid',
    baseDemucs_out+'drums.mid',
    baseDemucs_out+'other.mid',
    # baseDemucs_out+'piano.mid',
    baseDemucs_out+'vocals.mid'
]

try:
    for i, o in zip(input_paths, output_paths):
        model_output, midi_data, note_events = predict(
            i,
            model_or_model_path=str(ICASSP_2022_MODEL_PATH)
        )
        midi_data.write(o)
except Exception as e:
    print(e)

# spleeter

# base = '/content/mafia/mafia/'
baseSpleeter = base + 'spleeter/' + song_name + '/'
os.mkdir(base + 'spleeter_basicPitch')
baseSpleeter_out = base + 'spleeter_basicPitch/'

input_paths = [
    baseSpleeter+'bass.wav',
    baseSpleeter+'drums.wav',
    baseSpleeter+'other.wav',
    baseSpleeter+'piano.wav',
    baseSpleeter+'vocals.wav'
]

output_paths = [
    baseSpleeter_out+'bass.mid',
    baseSpleeter_out+'drums.mid',
    baseSpleeter_out+'other.mid',
    baseSpleeter_out+'piano.mid',
    baseSpleeter_out+'vocals.mid'
]

try:
    for i, o in zip(input_paths, output_paths):
        model_output, midi_data, note_events = predict(
            i,
            model_or_model_path=str(ICASSP_2022_MODEL_PATH)
        )
        midi_data.write(o)
except Exception as e:
    print(e)

# demucs_into_spleeter

# base = '/content/mafia/mafia/'
baseDemucsSpleeter = base + 'demucs_into_spleeter/other'  + '/'
os.mkdir(base + 'demucs_into_spleeter_basicPitch')
baseDemucsSpleeter_out = base + 'demucs_into_spleeter_basicPitch/'

input_paths = [
    baseDemucsSpleeter+'bass.wav',
    baseDemucsSpleeter+'drums.wav',
    baseDemucsSpleeter+'other.wav',
    baseDemucsSpleeter+'vocals.wav',
    baseDemucsSpleeter+'piano.wav'
]

output_paths = [
    baseDemucsSpleeter_out+'bass.mid',
    baseDemucsSpleeter_out+'drums.mid',
    baseDemucsSpleeter_out+'other.mid',
    baseDemucsSpleeter_out+'vocals.mid',
    baseDemucsSpleeter_out+'piano.mid'
]

try:
    for i, o in zip(input_paths, output_paths):
        model_output, midi_data, note_events = predict(
            i,
            model_or_model_path=str(ICASSP_2022_MODEL_PATH)
        )
        midi_data.write(o)
except Exception as e:
    print(e)
'''
with open('basic_pitch_execute.py', 'w') as basic_pitch_file:
  basic_pitch_file.write(string)
  basic_pitch_file.close()

In [ ]:
#@title Spotify Basic Pitch Transcription

!source split/bin/activate; split/bin/python 'basic_pitch_execute.py' $input_file_name $output_folder

In [ ]:
# Copyright 2021 Google LLC. All Rights Reserved.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

#@title MT3 Setup Environment
#@markdown Install MT3 and its dependencies (may take a few minutes).

!apt-get update -qq && apt-get install -qq libfluidsynth3 build-essential libasound2-dev libjack-dev

# install mt3
!git clone --branch=main https://github.com/magenta/mt3
!mv mt3 mt3_tmp; mv mt3_tmp/* .; rm -r mt3_tmp
!python3 -m pip install jax[cuda11_local] nest-asyncio pyfluidsynth==1.3.0 -e . -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# copy checkpoints
!gsutil -q -m cp -r gs://mt3/checkpoints .

# copy soundfont (originally from https://sites.google.com/site/soundfonts4u)
!gsutil -q -m cp gs://magentadata/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 .

import json
import IPython

# The below functions (load_gtag and log_event) handle Google Analytics event
# logging. The logging is anonymous and stores only very basic statistics of the
# audio and transcription e.g. length of audio, number of transcribed notes.

def load_gtag():
  """Loads gtag.js."""
  # Note: gtag.js MUST be loaded in the same cell execution as the one doing
  # synthesis. It does NOT persist across cell executions!
  html_code = '''
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-4P250YRJ08"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());
  gtag('config', 'G-4P250YRJ08',
       {'referrer': document.referrer.split('?')[0],
        'anonymize_ip': true,
        'page_title': '',
        'page_referrer': '',
        'cookie_prefix': 'magenta',
        'cookie_domain': 'auto',
        'cookie_expires': 0,
        'cookie_flags': 'SameSite=None;Secure'});
</script>
'''
  IPython.display.display(IPython.display.HTML(html_code))

def log_event(event_name, event_details):
  """Log event with name and details dictionary."""
  details_json = json.dumps(event_details)
  js_string = "gtag('event', '%s', %s);" % (event_name, details_json)
  IPython.display.display(IPython.display.Javascript(js_string))

load_gtag()
log_event('setupComplete', {})

In [ ]:
#@title MT3 Imports and Definitions

import functools
import os

import numpy as np
import tensorflow.compat.v2 as tf

import functools
import gin
import jax
import librosa
import note_seq
import seqio
import t5
import t5x

from mt3 import metrics_utils
from mt3 import models
from mt3 import network
from mt3 import note_sequences
from mt3 import preprocessors
from mt3 import spectrograms
from mt3 import vocabularies
from urllib.request import urlretrieve

from google.colab import files

import nest_asyncio
nest_asyncio.apply()

SAMPLE_RATE = 16000
SF2_PATH = 'SGM-v2.01-Sal-Guit-Bass-V1.3.sf2'

# DELETED ORIGINAL UPLOAD_FILE

def upload_audio_file(filename):
  return note_seq.audio_io.wav_data_to_samples_librosa(
    filename, SAMPLE_RATE)


class InferenceModel(object):
  """Wrapper of T5X model for music transcription."""

  def __init__(self, checkpoint_path, model_type='mt3'):

    # Model Constants.
    if model_type == 'ismir2021':
      num_velocity_bins = 127
      self.encoding_spec = note_sequences.NoteEncodingSpec
      self.inputs_length = 512
    elif model_type == 'mt3':
      num_velocity_bins = 1
      self.encoding_spec = note_sequences.NoteEncodingWithTiesSpec
      self.inputs_length = 256
    else:
      raise ValueError('unknown model_type: %s' % model_type)

    gin_files = ['/content/mt3/gin/model.gin',
                 f'/content/mt3/gin/{model_type}.gin']

    self.batch_size = 8
    self.outputs_length = 1024
    self.sequence_length = {'inputs': self.inputs_length,
                            'targets': self.outputs_length}

    self.partitioner = t5x.partitioning.PjitPartitioner(
        num_partitions=1)

    # Build Codecs and Vocabularies.
    self.spectrogram_config = spectrograms.SpectrogramConfig()
    self.codec = vocabularies.build_codec(
        vocab_config=vocabularies.VocabularyConfig(
            num_velocity_bins=num_velocity_bins))
    self.vocabulary = vocabularies.vocabulary_from_codec(self.codec)
    self.output_features = {
        'inputs': seqio.ContinuousFeature(dtype=tf.float32, rank=2),
        'targets': seqio.Feature(vocabulary=self.vocabulary),
    }

    # Create a T5X model.
    self._parse_gin(gin_files)
    self.model = self._load_model()

    # Restore from checkpoint.
    self.restore_from_checkpoint(checkpoint_path)

  @property
  def input_shapes(self):
    return {
          'encoder_input_tokens': (self.batch_size, self.inputs_length),
          'decoder_input_tokens': (self.batch_size, self.outputs_length)
    }

  def _parse_gin(self, gin_files):
    """Parse gin files used to train the model."""
    gin_bindings = [
        'from __gin__ import dynamic_registration',
        'from mt3 import vocabularies',
        'VOCAB_CONFIG=@vocabularies.VocabularyConfig()',
        'vocabularies.VocabularyConfig.num_velocity_bins=%NUM_VELOCITY_BINS'
    ]
    with gin.unlock_config():
      gin.parse_config_files_and_bindings(
          gin_files, gin_bindings, finalize_config=False)

  def _load_model(self):
    """Load up a T5X `Model` after parsing training gin config."""
    model_config = gin.get_configurable(network.T5Config)()
    module = network.Transformer(config=model_config)
    return models.ContinuousInputsEncoderDecoderModel(
        module=module,
        input_vocabulary=self.output_features['inputs'].vocabulary,
        output_vocabulary=self.output_features['targets'].vocabulary,
        optimizer_def=t5x.adafactor.Adafactor(decay_rate=0.8, step_offset=0),
        input_depth=spectrograms.input_depth(self.spectrogram_config))


  def restore_from_checkpoint(self, checkpoint_path):
    """Restore training state from checkpoint, resets self._predict_fn()."""
    train_state_initializer = t5x.utils.TrainStateInitializer(
      optimizer_def=self.model.optimizer_def,
      init_fn=self.model.get_initial_variables,
      input_shapes=self.input_shapes,
      partitioner=self.partitioner)

    restore_checkpoint_cfg = t5x.utils.RestoreCheckpointConfig(
        path=checkpoint_path, mode='specific', dtype='float32')

    train_state_axes = train_state_initializer.train_state_axes
    self._predict_fn = self._get_predict_fn(train_state_axes)
    self._train_state = train_state_initializer.from_checkpoint_or_scratch(
        [restore_checkpoint_cfg], init_rng=jax.random.PRNGKey(0))

  @functools.lru_cache()
  def _get_predict_fn(self, train_state_axes):
    """Generate a partitioned prediction function for decoding."""
    def partial_predict_fn(params, batch, decode_rng):
      return self.model.predict_batch_with_aux(
          params, batch, decoder_params={'decode_rng': None})
    return self.partitioner.partition(
        partial_predict_fn,
        in_axis_resources=(
            train_state_axes.params,
            t5x.partitioning.PartitionSpec('data',), None),
        out_axis_resources=t5x.partitioning.PartitionSpec('data',)
    )

  def predict_tokens(self, batch, seed=0):
    """Predict tokens from preprocessed dataset batch."""
    prediction, _ = self._predict_fn(
        self._train_state.params, batch, jax.random.PRNGKey(seed))
    return self.vocabulary.decode_tf(prediction).numpy()

  def __call__(self, audio):
    """Infer note sequence from audio samples.

    Args:
      audio: 1-d numpy array of audio samples (16kHz) for a single example.

    Returns:
      A note_sequence of the transcribed audio.
    """
    ds = self.audio_to_dataset(audio)
    ds = self.preprocess(ds)

    model_ds = self.model.FEATURE_CONVERTER_CLS(pack=False)(
        ds, task_feature_lengths=self.sequence_length)
    model_ds = model_ds.batch(self.batch_size)

    inferences = (tokens for batch in model_ds.as_numpy_iterator()
                  for tokens in self.predict_tokens(batch))

    predictions = []
    for example, tokens in zip(ds.as_numpy_iterator(), inferences):
      predictions.append(self.postprocess(tokens, example))

    result = metrics_utils.event_predictions_to_ns(
        predictions, codec=self.codec, encoding_spec=self.encoding_spec)
    return result['est_ns']

  def audio_to_dataset(self, audio):
    """Create a TF Dataset of spectrograms from input audio."""
    frames, frame_times = self._audio_to_frames(audio)
    return tf.data.Dataset.from_tensors({
        'inputs': frames,
        'input_times': frame_times,
    })

  def _audio_to_frames(self, audio):
    """Compute spectrogram frames from audio."""
    frame_size = self.spectrogram_config.hop_width
    padding = [0, frame_size - len(audio) % frame_size]
    audio = np.pad(audio, padding, mode='constant')
    frames = spectrograms.split_audio(audio, self.spectrogram_config)
    num_frames = len(audio) // frame_size
    times = np.arange(num_frames) / self.spectrogram_config.frames_per_second
    return frames, times

  def preprocess(self, ds):
    pp_chain = [
        functools.partial(
            t5.data.preprocessors.split_tokens_to_inputs_length,
            sequence_length=self.sequence_length,
            output_features=self.output_features,
            feature_key='inputs',
            additional_feature_keys=['input_times']),
        # Cache occurs here during training.
        preprocessors.add_dummy_targets,
        functools.partial(
            preprocessors.compute_spectrograms,
            spectrogram_config=self.spectrogram_config)
    ]
    for pp in pp_chain:
      ds = pp(ds)
    return ds

  def postprocess(self, tokens, example):
    tokens = self._trim_eos(tokens)
    start_time = example['input_times'][0]
    # Round down to nearest symbolic token step.
    start_time -= start_time % (1 / self.codec.steps_per_second)
    return {
        'est_tokens': tokens,
        'start_time': start_time,
        # Internal MT3 code expects raw inputs, not used here.
        'raw_inputs': []
    }

  @staticmethod
  def _trim_eos(tokens):
    tokens = np.array(tokens, np.int32)
    if vocabularies.DECODED_EOS_ID in tokens:
      tokens = tokens[:np.argmax(tokens == vocabularies.DECODED_EOS_ID)]
    return tokens



In [ ]:
#@title Load MT3 Model
#@markdown The `ismir2021` model transcribes piano only, with note velocities.
#@markdown The `mt3` model transcribes multiple simultaneous instruments,
#@markdown but without velocities.

MODEL = "mt3" #@param["ismir2021", "mt3"]

checkpoint_path = f'/content/checkpoints/{MODEL}/'

load_gtag()

log_event('loadModelStart', {'event_category': MODEL})
inference_model = InferenceModel(checkpoint_path, MODEL)
log_event('loadModelComplete', {'event_category': MODEL})


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title MT3 Transcription

### Demucs Files ###
load_gtag()
wav_files = []
directory = output_folder + '/htdemucs'
print(directory)
for root, dirs, file_list in os.walk(directory):
    for file in file_list:
        # Check the extension of files
        if file.endswith('.wav'):
            # Add the file to the list
            wav_files.append(os.path.join(root, file))
print(wav_files)
SAMPLE_RATE = 16000
transcribed_path = output_folder + "/demucs_mt3"
if not os.path.exists(transcribed_path):
  os.mkdir(transcribed_path)

for idx, wav_file in enumerate(wav_files):
  with open(wav_file, "rb") as in_file:
    log_event('uploadAudioStart', {})
    audio = upload_audio_file(in_file.read())
    log_event('uploadAudioComplete', {'value': round(len(audio) / SAMPLE_RATE)})

    note_seq.notebook_utils.colab_play(audio, sample_rate=SAMPLE_RATE)

    ### TRANSCRIBE ###

    log_event('transcribeStart', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE)
    })

    est_ns = inference_model(audio)

    log_event('transcribeComplete', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE),
        'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
        'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
        'numPrograms': len(set(note.program for note in est_ns.notes
                              if not note.is_drum))
    })

    note_seq.play_sequence(est_ns, synth=note_seq.fluidsynth,
                          sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
    # note_seq.plot_sequence(est_ns)

    ### DOWNLOAD ###

    #@title Download MIDI Transcription

    load_gtag()
    log_event('downloadTranscription', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE),
        'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
        'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
        'numPrograms': len(set(note.program for note in est_ns.notes
                              if not note.is_drum))
    })
    file_name = transcribed_path + "/" + wav_file.split('.')[0].split('/')[-1] + ".mid"
    note_seq.sequence_proto_to_midi_file(est_ns, file_name)
    # file_name = wav_file + "transcribed.mid"
    # urlretrieve('/tmp/transcribed.mid', file_name)


### Spleeter Files ###

load_gtag()
wav_files = []
directory = output_folder + '/spleeter'
print(directory)
for root, dirs, file_list in os.walk(directory):
    for file in file_list:
        # Check the extension of files
        if file.endswith('.wav'):
            # Add the file to the list
            wav_files.append(os.path.join(root, file))
print(wav_files)
SAMPLE_RATE = 16000
transcribed_path = output_folder + "/spleeter_mt3"
if not os.path.exists(transcribed_path):
  os.mkdir(transcribed_path)

for idx, wav_file in enumerate(wav_files):
  with open(wav_file, "rb") as in_file:
    log_event('uploadAudioStart', {})
    audio = upload_audio_file(in_file.read())
    log_event('uploadAudioComplete', {'value': round(len(audio) / SAMPLE_RATE)})

    note_seq.notebook_utils.colab_play(audio, sample_rate=SAMPLE_RATE)

    ### TRANSCRIBE ###

    log_event('transcribeStart', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE)
    })

    est_ns = inference_model(audio)

    log_event('transcribeComplete', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE),
        'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
        'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
        'numPrograms': len(set(note.program for note in est_ns.notes
                              if not note.is_drum))
    })

    note_seq.play_sequence(est_ns, synth=note_seq.fluidsynth,
                          sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
    # note_seq.plot_sequence(est_ns)

    ### DOWNLOAD ###

    #@title Download MIDI Transcription

    load_gtag()
    log_event('downloadTranscription', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE),
        'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
        'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
        'numPrograms': len(set(note.program for note in est_ns.notes
                              if not note.is_drum))
    })
    file_name = transcribed_path + "/" + wav_file.split('.')[0].split('/')[-1] + ".mid"
    note_seq.sequence_proto_to_midi_file(est_ns, file_name)
    # file_name = wav_file + "transcribed.mid"
    # urlretrieve('/tmp/transcribed.mid', file_name)

### MT3 Files ###
load_gtag()
wav_files = []
# IF NOT FROM SAME SESSION, REUPLOAD FILE ON THE SIDE (DO NOT USE ANY CELLS)
# nameFile = input_file + ".wav"
nameFile = input_file + '.mp3'
wav_files.append(nameFile)
# print(wav_files)
SAMPLE_RATE = 16000
transcribed_path = output_folder + "/mt3_only"
if not os.path.exists(transcribed_path):
  os.mkdir(transcribed_path)

for idx, wav_file in enumerate(wav_files):
  with open(wav_file, "rb") as in_file:
    log_event('uploadAudioStart', {})
    audio = upload_audio_file(in_file.read())
    log_event('uploadAudioComplete', {'value': round(len(audio) / SAMPLE_RATE)})

    note_seq.notebook_utils.colab_play(audio, sample_rate=SAMPLE_RATE)

    ### TRANSCRIBE ###

    log_event('transcribeStart', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE)
    })

    est_ns = inference_model(audio)

    log_event('transcribeComplete', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE),
        'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
        'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
        'numPrograms': len(set(note.program for note in est_ns.notes
                              if not note.is_drum))
    })

    note_seq.play_sequence(est_ns, synth=note_seq.fluidsynth,
                          sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
    # note_seq.plot_sequence(est_ns)

    ### DOWNLOAD ###

    #@title Download MIDI Transcription

    load_gtag()
    log_event('downloadTranscription', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE),
        'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
        'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
        'numPrograms': len(set(note.program for note in est_ns.notes
                              if not note.is_drum))
    })
    file_name = transcribed_path + "/" + wav_file.split('.')[0].split('/')[-1] + ".mid"
    note_seq.sequence_proto_to_midi_file(est_ns, file_name)
    # file_name = wav_file + "transcribed.mid"
    # urlretrieve('/tmp/transcribed.mid', file_name)


### Demucs into Spleeter Files ###
load_gtag()
wav_files = []
directory = output_folder + '/demucs_into_spleeter/other'
print(directory)
for root, dirs, file_list in os.walk(directory):
    for file in file_list:
        # Check the extension of files
        if file.endswith('.wav'):
            # Add the file to the list
            wav_files.append(os.path.join(root, file))
print(wav_files)
SAMPLE_RATE = 16000
transcribed_path = output_folder + "/demucs_into_spleeter_mt3"
if not os.path.exists(transcribed_path):
  os.mkdir(transcribed_path)

for idx, wav_file in enumerate(wav_files):
  with open(wav_file, "rb") as in_file:
    log_event('uploadAudioStart', {})
    audio = upload_audio_file(in_file.read())
    log_event('uploadAudioComplete', {'value': round(len(audio) / SAMPLE_RATE)})

    note_seq.notebook_utils.colab_play(audio, sample_rate=SAMPLE_RATE)

    ### TRANSCRIBE ###

    log_event('transcribeStart', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE)
    })

    est_ns = inference_model(audio)

    log_event('transcribeComplete', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE),
        'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
        'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
        'numPrograms': len(set(note.program for note in est_ns.notes
                              if not note.is_drum))
    })

    note_seq.play_sequence(est_ns, synth=note_seq.fluidsynth,
                          sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
    # note_seq.plot_sequence(est_ns)

    ### DOWNLOAD ###

    #@title Download MIDI Transcription

    load_gtag()
    log_event('downloadTranscription', {
        'event_category': MODEL,
        'value': round(len(audio) / SAMPLE_RATE),
        'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
        'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
        'numPrograms': len(set(note.program for note in est_ns.notes
                              if not note.is_drum))
    })
    file_name = transcribed_path + "/" + wav_file.split('.')[0].split('/')[-1] + ".mid"
    note_seq.sequence_proto_to_midi_file(est_ns, file_name)
    # file_name = wav_file + "transcribed.mid"
    # urlretrieve('/tmp/transcribed.mid', file_name)

In [ ]:
#@title Pipeline Lists

#1 MT3 only
#2 Demucs + Spleeter -> MT3
#3 Spleeter -> MT3
#4 Demucs + Spleeter -> Basic Pitch
#5 Spleeter -> Basic Pitch


In [ ]:
#@title MIDI Combiner imports and functions
!pip install pretty_midi
from pretty_midi import PrettyMIDI
from pathlib import Path

stem_info = {
    'bass': (33, False, 'bass'),
    'drums': (114, True, 'drums'),
    'piano': (2, False, 'piano'),
    'other': (88, False, 'other')
}

def get_updated_midi(p):
    updatedMidi = PrettyMIDI(p)
    instr_type = Path(p).stem

    for instr in updatedMidi.instruments:
        instr.program = stem_info[instr_type][0]
        instr.is_drum = stem_info[instr_type][1]
        instr.name = stem_info[instr_type][2]

    # print((updatedMidi.instruments[0].program, updatedMidi.instruments[0].is_drum, updatedMidi.instruments[0].name))

    return updatedMidi


#######################################
def get_removed_instruments(old_insts, new_insts):
    removed_insts = []
    for inst in old_insts:
        if inst not in new_insts:
            removed_insts.append(inst)
    return removed_insts

def remove_drums_from_stem(midi):
    '''
    Removes any instrument from `midi` that falls inside the drums instrument class.

        Parameters:
            midi (pretty_midi.PrettyMIDI): PrettyMIDI object to remove drums from.
    '''
    # input should be anything that is NOT the drum stem
    new_insts_list = list(filter(lambda inst: not inst.is_drum, midi.instruments))
    removed_insts = get_removed_instruments(midi.instruments, new_insts_list)
    midi.instruments = new_insts_list
    return removed_insts

def remove_bass_from_stem(midi):
    '''
    Removes any instrument from `midi` that falls inside the bass instrument class.

        Parameters:
            midi (pretty_midi.PrettyMIDI): PrettyMIDI object to remove bass from.
    '''
    # input should be anything that is NOT the bass stem
    new_insts_list = list(filter(lambda inst: inst.program not in range(32, 40), midi.instruments))
    removed_insts = get_removed_instruments(midi.instruments, new_insts_list)
    midi.instruments = new_insts_list
    return removed_insts

def remove_piano_from_stem(midi):
    '''
    Removes any instrument from `midi` that falls inside the piano instrument class.

        Parameters:
            midi (pretty_midi.PrettyMIDI): PrettyMIDI object to remove pianos from.
    '''
    # input should be anything that is NOT the piano stem
    new_insts_list = list(filter(lambda inst: inst.program not in range(8), midi.instruments))
    removed_insts = get_removed_instruments(midi.instruments, new_insts_list)
    midi.instruments = new_insts_list
    return removed_insts

def remove_other_from_stem(midi):
    '''
    Removes any instrument from `midi` that does not fall inside bass, drums, or piano instrument classes.

        Parameters:
            midi (pretty_midi.PrettyMIDI): PrettyMIDI object to remove `other` instruments from.
    '''
    # input should be anything that is NOT the other stem
    def predicate(inst):
        return \
            inst.program in range(8) or \
            inst.program in range(33, 40) or \
            inst.is_drum

    new_insts_list = list(filter(
        predicate,
        midi.instruments
    ))
    removed_insts = get_removed_instruments(midi.instruments, new_insts_list)
    midi.instruments = new_insts_list
    return removed_insts

def insert_insts_notes(midi, insts):
    '''
    For each matching instrument in `midi.instrument` and `insts`, transfer the instrument's notes from `insts`
    to the corresponding instrument in `midi.instrument`.  For each instrument in `insts` that is not in `midi.instruments`,
    simply add it to `midi.instruments`.
    '''
    num_notes_in_old_midi = 0
    for i in midi.instruments:
        for n in i.notes:
            num_notes_in_old_midi += 1

    num_notes_in_insts = 0
    for i in insts:
        for n in i.notes:
            num_notes_in_insts += 1

    m_insts_program_nums = [m_inst.program for m_inst in midi.instruments]
    for inst in insts:
        if inst.program in m_insts_program_nums:
            for m_inst in midi.instruments:
                if inst.program == m_inst.program:
                    for note in inst.notes:
                        m_inst.notes.append(note)
                    m_inst.notes.sort(key=lambda n: n.start)
                    break
        else:
            midi.instruments.append(inst)

    num_notes_in_new_midi = 0
    for i in midi.instruments:
        for n in i.notes:
            num_notes_in_new_midi += 1


def clean_stem(midi, other_midis, stem):
    '''
    Removes unnecessary instruments from `midi` depending on what type of `stem` it is.

        Parameters:
            midi (pretty_midi.PrettyMIDI): PrettyMIDI object to clean.
            stem (str): string specifying the stem type--only accepts 'piano', 'drums', 'bass', and 'other'.
    '''

    PIANO = 'piano'
    DRUMS = 'drums'
    BASS = 'bass'
    OTHER = 'other'

    if stem == PIANO:
        extra_bass = remove_bass_from_stem(midi)
        extra_drums = remove_drums_from_stem(midi)
        extra_other = remove_other_from_stem(midi)

        insert_insts_notes(other_midis['bass'], extra_bass)
        insert_insts_notes(other_midis['drums'], extra_drums)
        insert_insts_notes(other_midis['other'], extra_other)

    elif stem == DRUMS:
        extra_piano = remove_piano_from_stem(midi)
        extra_bass = remove_bass_from_stem(midi)
        extra_other = remove_other_from_stem(midi)

        insert_insts_notes(other_midis['piano'], extra_piano)
        insert_insts_notes(other_midis['bass'], extra_bass)
        insert_insts_notes(other_midis['other'], extra_other)

    elif stem == BASS:
        extra_piano = remove_piano_from_stem(midi)
        extra_drums = remove_drums_from_stem(midi)
        extra_other = remove_other_from_stem(midi)

        insert_insts_notes(other_midis['piano'], extra_piano)
        insert_insts_notes(other_midis['drums'], extra_drums)
        insert_insts_notes(other_midis['other'], extra_other)

    elif stem == OTHER:
        extra_bass = remove_bass_from_stem(midi)
        extra_drums = remove_drums_from_stem(midi)
        extra_piano = remove_piano_from_stem(midi)

        insert_insts_notes(other_midis['bass'], extra_bass)
        insert_insts_notes(other_midis['drums'], extra_drums)
        insert_insts_notes(other_midis['piano'], extra_piano)

    else:
        raise Exception(f'invalid stem: {stem}')
#######################################

In [ ]:
#@title Pipeline #1 - MT3 Only

pipeline1 = output_folder + "/pipeline1"

# if not os.path.exists(pipeline1):
#   os.mkdir(pipeline1)

mt3_only = output_folder + "/mt3_only"

!mkdir $pipeline1
!cp -r $mt3_only $pipeline1


In [ ]:
#@title Combining MIDI - Pipeline #2 Demucs + Spleeter -> MT3

piano = output_folder + "/demucs_into_spleeter_mt3/piano.mid"
drums = output_folder + "/demucs_mt3/drums.mid"
bass = output_folder + "/demucs_mt3/bass.mid"
other = output_folder + "/demucs_into_spleeter_mt3/other.mid"

pipeline2 = output_folder + "/pipeline2"
pipeline2_selectedmid = pipeline2 + "/selectedmid"

!mkdir $pipeline2
!mkdir $pipeline2_selectedmid
!cp $piano $pipeline2_selectedmid
!cp $drums $pipeline2_selectedmid
!cp $other $pipeline2_selectedmid
!cp $bass $pipeline2_selectedmid

paths = []

# midi input file path is /content/song_files/{pipeline_name}/*
    # pipeline_name will be provided via command line args
    # where pipeline_name is in {spleeter, demucs, mt3, basic_pitch}
    # where '*' represents each midi file in /content/[pipeline_name]/

# pipeline_name = pipeline2

# loop the /content/song_files/{pipeline_name}/ folder and append path of each midi file into it
target_folder = pipeline2_selectedmid
for midi_filename in os.listdir(target_folder):
    f = os.path.join(target_folder, midi_filename)
    # checking if it is a file
    if os.path.isfile(f):
        paths.append(f)

#######################################
stem_to_midi = dict()
for p in paths:
    stem_to_midi[Path(p).stem] = PrettyMIDI(p)

for p in paths:
    stem = Path(p).stem
    midi = stem_to_midi.pop(stem)
    clean_stem(midi, stem_to_midi, stem)
    stem_to_midi[stem] = midi
    pass

for stem in stem_to_midi:
    midi = stem_to_midi[stem]
    for i in midi.instruments:
        needs_sorting = False
        for j in range(len(i.notes) - 1):
            if i.notes[j].start > i.notes[j + 1].start:
                needs_sorting = True
        if needs_sorting:
            i.notes = sorted(i.notes, key=lambda note: note.start)
    midi.write(f'{target_folder}/{stem}.mid')
#######################################

# Load MIDI file into PrettyMIDI object
midi_tracks = [get_updated_midi(p) for p in paths]

combined_midi = PrettyMIDI()

for track in midi_tracks:
    for instrument in track.instruments:
        combined_midi.instruments.append(instrument)

midi_tracks_for_eval = [PrettyMIDI(p) for p in paths]

combined_midi_eval = PrettyMIDI()

for track in midi_tracks_for_eval:
    for instrument in track.instruments:
        combined_midi_eval.instruments.append(instrument)

combined_midi_eval.write(f'{pipeline2}/pipeline2_combined_midi_eval.mid')

# write to ./content/song_files/combined_midi_output/combined_midi.mid
combined_path = pipeline2 + '/pipeline2_combined_midi.mid'
combined_midi.write(combined_path)

In [ ]:
#@title Combining MIDI - Pipeline #3 Spleeter -> MT3

piano = output_folder + "/spleeter_mt3/piano.mid"
drums = output_folder + "/spleeter_mt3/drums.mid"
bass = output_folder + "/spleeter_mt3/bass.mid"
other = output_folder + "/spleeter_mt3/other.mid"

pipeline3 = output_folder + "/pipeline3"
pipeline3_selectedmid = pipeline3 + "/selectedmid"

!mkdir $pipeline3
!mkdir $pipeline3_selectedmid
!cp $drums $pipeline3_selectedmid
!cp $other $pipeline3_selectedmid
!cp $bass $pipeline3_selectedmid
!cp $piano $pipeline3_selectedmid

paths = []

# midi input file path is /content/song_files/{pipeline_name}/*
    # pipeline_name will be provided via command line args
    # where pipeline_name is in {spleeter, demucs, mt3, basic_pitch}
    # where '*' represents each midi file in /content/[pipeline_name]/

# pipeline_name = pipeline3

# loop the /content/song_files/{pipeline_name}/ folder and append path of each midi file into it
target_folder = pipeline3_selectedmid
for midi_filename in os.listdir(target_folder):
    f = os.path.join(target_folder, midi_filename)
    # checking if it is a file
    if os.path.isfile(f):
        paths.append(f)

#######################################
stem_to_midi = dict()
for p in paths:
    stem_to_midi[Path(p).stem] = PrettyMIDI(p)

for p in paths:
    stem = Path(p).stem
    midi = stem_to_midi.pop(stem)
    clean_stem(midi, stem_to_midi, stem)
    stem_to_midi[stem] = midi
    pass

for stem in stem_to_midi:
    midi = stem_to_midi[stem]
    for i in midi.instruments:
        needs_sorting = False
        for j in range(len(i.notes) - 1):
            if i.notes[j].start > i.notes[j + 1].start:
                needs_sorting = True
        if needs_sorting:
            i.notes = sorted(i.notes, key=lambda note: note.start)
    midi.write(f'{target_folder}/{stem}.mid')
#######################################

# Load MIDI file into PrettyMIDI object
midi_tracks = [get_updated_midi(p) for p in paths] # [PrettyMIDI(p) for p in paths]

combined_midi = PrettyMIDI()

for track in midi_tracks:
    for instrument in track.instruments:
        combined_midi.instruments.append(instrument)

midi_tracks_for_eval = [PrettyMIDI(p) for p in paths]

combined_midi_eval = PrettyMIDI()

for track in midi_tracks_for_eval:
    for instrument in track.instruments:
        combined_midi_eval.instruments.append(instrument)

combined_midi_eval.write(f'{pipeline3}/pipeline3_combined_midi_eval.mid')

# write to ./content/song_files/combined_midi_output/combined_midi.mid
combined_path = pipeline3 + '/pipeline3_combined_midi.mid'
combined_midi.write(combined_path)

In [ ]:
#@title Combining MIDI - Pipeline #4 Demucs + Spleeter -> Basic Pitch

piano = output_folder + "/demucs_into_spleeter_basicPitch/piano.mid"
drums = output_folder + "/demucs_basicPitch/drums.mid"
bass = output_folder + "/demucs_basicPitch/bass.mid"
other = output_folder + "/demucs_into_spleeter_basicPitch/other.mid"

pipeline4 = output_folder + "/pipeline4"
pipeline4_selectedmid = pipeline4 + "/selectedmid"

!mkdir $pipeline4
!mkdir $pipeline4_selectedmid
!cp $piano $pipeline4_selectedmid
!cp $drums $pipeline4_selectedmid
!cp $other $pipeline4_selectedmid
!cp $bass $pipeline4_selectedmid

paths = []

# midi input file path is /content/song_files/{pipeline_name}/*
    # pipeline_name will be provided via command line args
    # where pipeline_name is in {spleeter, demucs, mt3, basic_pitch}
    # where '*' represents each midi file in /content/[pipeline_name]/

# pipeline_name = pipeline4

# loop the /content/song_files/{pipeline_name}/ folder and append path of each midi file into it
target_folder = pipeline4_selectedmid
for midi_filename in os.listdir(target_folder):
    f = os.path.join(target_folder, midi_filename)
    # checking if it is a file
    if os.path.isfile(f):
        paths.append(f)

# Load MIDI file into PrettyMIDI object
midi_tracks = [get_updated_midi(p) for p in paths]

combined_midi = PrettyMIDI()

for track in midi_tracks:
    for instrument in track.instruments:
        combined_midi.instruments.append(instrument)

# write to ./content/song_files/combined_midi_output/combined_midi.mid
combined_path = pipeline4 + '/pipeline4_combined_midi.mid'
combined_midi.write(combined_path)

In [ ]:
#@title Combining MIDI - Pipeline #5 Spleeter -> Basic Pitch

piano = output_folder + "/spleeter_basicPitch/piano.mid"
drums = output_folder + "/spleeter_basicPitch/drums.mid"
bass = output_folder + "/spleeter_basicPitch/bass.mid"
other = output_folder + "/spleeter_basicPitch/other.mid"

pipeline5 = output_folder + "/pipeline5"
pipeline5_selectedmid = pipeline5 + "/selectedmid"

!mkdir $pipeline5
!mkdir $pipeline5_selectedmid
!cp $piano $pipeline5_selectedmid
!cp $drums $pipeline5_selectedmid
!cp $other $pipeline5_selectedmid
!cp $bass $pipeline5_selectedmid

paths = []

# midi input file path is /content/song_files/{pipeline_name}/*
    # pipeline_name will be provided via command line args
    # where pipeline_name is in {spleeter, demucs, mt3, basic_pitch}
    # where '*' represents each midi file in /content/[pipeline_name]/

# pipeline_name = pipeline5

# loop the /content/song_files/{pipeline_name}/ folder and append path of each midi file into it
target_folder = pipeline5_selectedmid
for midi_filename in os.listdir(target_folder):
    f = os.path.join(target_folder, midi_filename)
    # checking if it is a file
    if os.path.isfile(f):
        paths.append(f)

# Load MIDI file into PrettyMIDI object
midi_tracks = [get_updated_midi(p) for p in paths]

combined_midi = PrettyMIDI()

for track in midi_tracks:
    for instrument in track.instruments:
        combined_midi.instruments.append(instrument)

# write to ./content/song_files/combined_midi_output/combined_midi.mid
combined_path = pipeline5 + '/pipeline5_combined_midi.mid'
combined_midi.write(combined_path)

In [ ]:
#@title MIDI to Noteblock Studio (NBS) Imports and Definitions
!pip install pretty_midi
!pip install pynbs
from pretty_midi import PrettyMIDI
import pynbs

midi_to_nbs_instrs = {
    0: 0, # piano
    2: 0, # piano
    114: 0, # drums
    33: 1, # bass
    78: 0, # vocals
    88: 0, # other
    40: 0, # guitar
    53: 0,
    61: 0,
    64: 0,
    78: 0
}

nbs_instr_files = {
    0: 'harp.ogg',
    3: 'sdrum.ogg',
    1: 'bdrum.ogg',
    6: 'flute.ogg',
    9: 'harp.ogg',
    5: 'guitar.ogg',
    8: 'icechime.ogg',
    15: 'pling.ogg',
    10: 'iron_xylophone.ogg'
}

nbs_instr_names = {
    0: 'harp',
    3: 'snare drum',
    1: 'bass drum',
    6: 'flute',
    9: 'harp',
    5: 'guitar',
    8: 'icechime',
    15: 'pling',
    10: 'iron xylophone'
}




In [ ]:
#@title Pipeline #1 MIDI to NBS Transcription
!pip install pretty_midi

from pretty_midi import PrettyMIDI
programNum_to_representativeNum = dict()

# Note: All Instruments we do not know have been matched to piano by defualt
# piano
for i in range(8):
    programNum_to_representativeNum[i] = 0 # grand piano represents piano

# chromatic percussion
for i in range(8, 16):
    programNum_to_representativeNum[i] = 114

# organ
for i in range(16, 24):
    programNum_to_representativeNum[i] = 0 # grand piano represents piano

# guitar
for i in range(24, 33):
    programNum_to_representativeNum[i] = 40 # violin represents strings

# bass
for i in range(32, 40):
    programNum_to_representativeNum[i] = 33 # acoustic bass

# strings
for i in range(40, 48):
    programNum_to_representativeNum[i] =  40 # violin represents strings

# ensemble
for i in range(48, 56):
    programNum_to_representativeNum[i] = 53 # choir aahs represents ensemble

# brass
for i in range(56, 64):
    programNum_to_representativeNum[i] = 61 # french horn represents brass

# reed
for i in range(64, 72):
    programNum_to_representativeNum[i] = 64 # soprano sax represents reed

# Pipe
for i in range(72, 80):
    programNum_to_representativeNum[i] = 78

# Synth Lead
for i in range(80, 88):
    programNum_to_representativeNum[i] = 0

# Synth Pad
for i in range(88, 96):
    programNum_to_representativeNum[i] = 0

# Synth Effects
for i in range(96, 104):
    programNum_to_representativeNum[i] = 0

# Ethnic
for i in range(104, 112):
    programNum_to_representativeNum[i] = 0

# Percussive
for i in range(112, 120):
    programNum_to_representativeNum[i] = 0

# Sound Effects
for i in range(120, 128):
    programNum_to_representativeNum[i] = 0

# get the midid file and loop through its instruments

# Load MIDI file into PrettyMIDI object (pass in file path of the combined midi)

pipeline1 =  output_folder + "/pipeline1/mt3_only/" + input_file + ".mid"
midi_file = PrettyMIDI(pipeline1)
for instrument in midi_file.instruments:
    instrument.program = programNum_to_representativeNum[instrument.program]

# get midi input from /content/song_files/combined_midi/combined_midi.mid

# in many songs (and thus midi files), notes are played at the same time (same onset time)
# in nbs, layers are used when multiple notes are played at the same time
# so:
    # for each instrument
        # and for each note in that instrument
            # get the first note and its onset time
            # get all other notes with the same onset time
            # for each of those notes
                # create an nbs note and append it to the list of notes
                # each note should be put in a different layer (since they are played at the same time)
            # update the maximum number of layers
            # remove the notes we just converted from the midi instrument notes list
        # update layer offset (as to not override notes we just did for the previous instrument)

list_of_notes = []
layer_offsets = [0]
nbs_instrument_set = set()
max_num_ticks = 0
for i, instrument in enumerate(midi_file.instruments):
    max_num_layers = 0
    while(len(instrument.notes) > 0):
        curr_note = instrument.notes[0]
        time = curr_note.start
        # Filter midi notes according to nearly similar start time
        notes_at_time = list(filter(lambda n: n.start < time + 0.05, instrument.notes))
        # Convert all filtered notes to NBS notes and store all layers of instrument i's notes
        # in layer_offsets
        for j in range(len(notes_at_time)):
            max_num_ticks = int(round(time / 0.05)) if max_num_ticks < int(round(time / 0.05)) else max_num_ticks
            n = notes_at_time[j]
            pynbs_instrument_id = midi_to_nbs_instrs[programNum_to_representativeNum[instrument.program]]
            list_of_notes.append(
                pynbs.Note(
                    tick=int(round(time / 0.05)),
                    layer=j + layer_offsets[i],
                    instrument=pynbs_instrument_id,
                    key=n.pitch - 21,
                    velocity=n.velocity,
                    pitch=0
                )
            )
            nbs_instrument_set.add(pynbs_instrument_id)
        max_num_layers = len(notes_at_time) if len(notes_at_time) > max_num_layers else max_num_layers
        # Get the leftover midi notes of instrument i and then sort them into layers in the next iteration;
        # keep at it until we run out of midi notes
        instrument.notes = [note for note in instrument.notes if note not in notes_at_time]
    # Once done with a single instrument, add offset for what layer instrument (i+1)'s notes will
    # start to be inserted
    layer_offsets.append(layer_offsets[i] + max_num_layers)

# sort the nbs notes by tick in ascending order--required for saving
list_of_notes = sorted(list_of_notes, key=lambda note: note.tick)

# Testing for conflicting notes in same layer
for tick in range(max_num_ticks + 1):
    notes_at_tick = [note for note in list_of_notes if note.tick == tick]
    for i in range(len(notes_at_tick)):
        for j in range(len(notes_at_tick)):
            if j != i:
                note_i_layer = notes_at_tick[i].layer
                note_j_layer = notes_at_tick[j].layer
                if note_i_layer == note_j_layer:
                    print(str(notes_at_tick[i]), str(notes_at_tick[j], 'have the same layer and tick'))
                    exit(-1)

nbs_file = pynbs.new_file(
    song_name='song',
    tempo=20.00,
    song_layers=layer_offsets[-1],
    song_length=max_num_ticks
)
nbs_file.layers = [pynbs.Layer(id=i, name='', lock=False, volume=100, panning=0) for i in range(layer_offsets[-1])]
nbs_file.instruments = [pynbs.Instrument(id=i, name=nbs_instr_names[i], file=nbs_instr_files[i], pitch=45, press_key=True) for i in nbs_instrument_set]
nbs_file.notes.extend(list_of_notes)
# write to nbs_output.nbs in /content/song_files/nbs_output/combined_nbs.nbs
output_location = output_folder + "/pipeline1/pipeline1.nbs"
nbs_file.save(output_location, version=5)

In [ ]:
#@title Pipeline #2 MIDI to NBS Transcription
# get midi input from /content/song_files/combined_midi/combined_midi.mid
!pip install pretty_midi

pipeline2 =  output_folder + "/pipeline2/pipeline2_combined_midi.mid"
midi_file = PrettyMIDI(pipeline2)

# in many songs (and thus midi files), notes are played at the same time (same onset time)
# in nbs, layers are used when multiple notes are played at the same time
# so:
    # for each instrument
        # and for each note in that instrument
            # get the first note and its onset time
            # get all other notes with the same onset time
            # for each of those notes
                # create an nbs note and append it to the list of notes
                # each note should be put in a different layer (since they are played at the same time)
            # update the maximum number of layers
            # remove the notes we just converted from the midi instrument notes list
        # update layer offset (as to not override notes we just did for the previous instrument)

list_of_notes = []
layer_offsets = [0]
nbs_instrument_set = set()
max_num_ticks = 0
for i, instrument in enumerate(midi_file.instruments):
    max_num_layers = 0
    while(len(instrument.notes) > 0):
        curr_note = instrument.notes[0]
        time = curr_note.start
        # Filter midi notes according to nearly similar start time
        notes_at_time = list(filter(lambda n: n.start < time + 0.05, instrument.notes))
        # Convert all filtered notes to NBS notes and store all layers of instrument i's notes
        # in layer_offsets
        for j in range(len(notes_at_time)):
            max_num_ticks = int(round(time / 0.05)) if max_num_ticks < int(round(time / 0.05)) else max_num_ticks
            n = notes_at_time[j]
            pynbs_instrument_id = midi_to_nbs_instrs[instrument.program]
            list_of_notes.append(
                pynbs.Note(
                    tick=int(round(time / 0.05)),
                    layer=j + layer_offsets[i],
                    instrument=pynbs_instrument_id,
                    key=n.pitch - 21 if n.pitch - 21 >= 0 else 0,
                    velocity=n.velocity,
                    pitch=0
                )
            )
            nbs_instrument_set.add(pynbs_instrument_id)
        max_num_layers = len(notes_at_time) if len(notes_at_time) > max_num_layers else max_num_layers
        # Get the leftover midi notes of instrument i and then sort them into layers in the next iteration;
        # keep at it until we run out of midi notes
        instrument.notes = [note for note in instrument.notes if note not in notes_at_time]
    # Once done with a single instrument, add offset for what layer instrument (i+1)'s notes will
    # start to be inserted
    layer_offsets.append(layer_offsets[i] + max_num_layers)

# sort the nbs notes by tick in ascending order--required for saving
list_of_notes = sorted(list_of_notes, key=lambda note: note.tick)

# Testing for conflicting notes in same layer
for tick in range(max_num_ticks + 1):
    notes_at_tick = [note for note in list_of_notes if note.tick == tick]
    for i in range(len(notes_at_tick)):
        for j in range(len(notes_at_tick)):
            if j != i:
                note_i_layer = notes_at_tick[i].layer
                note_j_layer = notes_at_tick[j].layer
                if note_i_layer == note_j_layer:
                    print(str(notes_at_tick[i]), str(notes_at_tick[j], 'have the same layer and tick'))
                    exit(-1)

nbs_file = pynbs.new_file(
    song_name='song',
    tempo=20.00,
    song_layers=layer_offsets[-1],
    song_length=max_num_ticks
)
nbs_file.layers = [pynbs.Layer(id=i, name='', lock=False, volume=100, panning=0) for i in range(layer_offsets[-1])]
nbs_file.instruments = [pynbs.Instrument(id=i, name=nbs_instr_names[i], file=nbs_instr_files[i], pitch=45, press_key=True) for i in nbs_instrument_set]
nbs_file.notes.extend(list_of_notes)
# write to nbs_output.nbs in /content/song_files/nbs_output/combined_nbs.nbs
output_location = output_folder + "/pipeline2/pipeline2.nbs"
nbs_file.save(output_location, version=5)

In [ ]:
#@title Pipeline #3 MIDI to NBS Transcription
# get midi input from /content/song_files/combined_midi/combined_midi.mid
!pip install pretty_midi

pipeline3 = output_folder + "/pipeline3/pipeline3_combined_midi.mid"
midi_file = PrettyMIDI(pipeline3)

# in many songs (and thus midi files), notes are played at the same time (same onset time)
# in nbs, layers are used when multiple notes are played at the same time
# so:
    # for each instrument
        # and for each note in that instrument
            # get the first note and its onset time
            # get all other notes with the same onset time
            # for each of those notes
                # create an nbs note and append it to the list of notes
                # each note should be put in a different layer (since they are played at the same time)
            # update the maximum number of layers
            # remove the notes we just converted from the midi instrument notes list
        # update layer offset (as to not override notes we just did for the previous instrument)

list_of_notes = []
layer_offsets = [0]
nbs_instrument_set = set()
max_num_ticks = 0
for i, instrument in enumerate(midi_file.instruments):
    max_num_layers = 0
    while(len(instrument.notes) > 0):
        curr_note = instrument.notes[0]
        time = curr_note.start
        # Filter midi notes according to nearly similar start time
        notes_at_time = list(filter(lambda n: n.start < time + 0.05, instrument.notes))
        # Convert all filtered notes to NBS notes and store all layers of instrument i's notes
        # in layer_offsets
        for j in range(len(notes_at_time)):
            max_num_ticks = int(round(time / 0.05)) if max_num_ticks < int(round(time / 0.05)) else max_num_ticks
            n = notes_at_time[j]
            pynbs_instrument_id = midi_to_nbs_instrs[instrument.program]
            list_of_notes.append(
                pynbs.Note(
                    tick=int(round(time / 0.05)),
                    layer=j + layer_offsets[i],
                    instrument=pynbs_instrument_id,
                    key=n.pitch - 21,
                    velocity=n.velocity,
                    pitch=0
                )
            )
            nbs_instrument_set.add(pynbs_instrument_id)
        max_num_layers = len(notes_at_time) if len(notes_at_time) > max_num_layers else max_num_layers
        # Get the leftover midi notes of instrument i and then sort them into layers in the next iteration;
        # keep at it until we run out of midi notes
        instrument.notes = [note for note in instrument.notes if note not in notes_at_time]
    # Once done with a single instrument, add offset for what layer instrument (i+1)'s notes will
    # start to be inserted
    layer_offsets.append(layer_offsets[i] + max_num_layers)

# sort the nbs notes by tick in ascending order--required for saving
list_of_notes = sorted(list_of_notes, key=lambda note: note.tick)

# Testing for conflicting notes in same layer
for tick in range(max_num_ticks + 1):
    notes_at_tick = [note for note in list_of_notes if note.tick == tick]
    for i in range(len(notes_at_tick)):
        for j in range(len(notes_at_tick)):
            if j != i:
                note_i_layer = notes_at_tick[i].layer
                note_j_layer = notes_at_tick[j].layer
                if note_i_layer == note_j_layer:
                    print(str(notes_at_tick[i]), str(notes_at_tick[j], 'have the same layer and tick'))
                    exit(-1)

nbs_file = pynbs.new_file(
    song_name='song',
    tempo=20.00,
    song_layers=layer_offsets[-1],
    song_length=max_num_ticks
)
nbs_file.layers = [pynbs.Layer(id=i, name='', lock=False, volume=100, panning=0) for i in range(layer_offsets[-1])]
nbs_file.instruments = [pynbs.Instrument(id=i, name=nbs_instr_names[i], file=nbs_instr_files[i], pitch=45, press_key=True) for i in nbs_instrument_set]
nbs_file.notes.extend(list_of_notes)
# write to nbs_output.nbs in /content/song_files/nbs_output/combined_nbs.nbs
output_location = output_folder + "/pipeline3/pipeline3.nbs"
nbs_file.save(output_location, version=5)

In [ ]:
#@title Pipeline #4 MIDI to NBS Transcription
# get midi input from /content/song_files/combined_midi/combined_midi.mid
!pip install pretty_midi

pipeline4 =  output_folder + "/pipeline4/pipeline4_combined_midi.mid"
midi_file = PrettyMIDI(pipeline4)

# in many songs (and thus midi files), notes are played at the same time (same onset time)
# in nbs, layers are used when multiple notes are played at the same time
# so:
    # for each instrument
        # and for each note in that instrument
            # get the first note and its onset time
            # get all other notes with the same onset time
            # for each of those notes
                # create an nbs note and append it to the list of notes
                # each note should be put in a different layer (since they are played at the same time)
            # update the maximum number of layers
            # remove the notes we just converted from the midi instrument notes list
        # update layer offset (as to not override notes we just did for the previous instrument)

list_of_notes = []
layer_offsets = [0]
nbs_instrument_set = set()
max_num_ticks = 0
for i, instrument in enumerate(midi_file.instruments):
    max_num_layers = 0
    while(len(instrument.notes) > 0):
        curr_note = instrument.notes[0]
        time = curr_note.start
        # Filter midi notes according to nearly similar start time
        notes_at_time = list(filter(lambda n: n.start < time + 0.05, instrument.notes))
        # Convert all filtered notes to NBS notes and store all layers of instrument i's notes
        # in layer_offsets
        for j in range(len(notes_at_time)):
            max_num_ticks = int(round(time / 0.05)) if max_num_ticks < int(round(time / 0.05)) else max_num_ticks
            n = notes_at_time[j]
            pynbs_instrument_id = midi_to_nbs_instrs[instrument.program]
            list_of_notes.append(
                pynbs.Note(
                    tick=int(round(time / 0.05)),
                    layer=j + layer_offsets[i],
                    instrument=pynbs_instrument_id,
                    key=n.pitch - 21,
                    velocity=n.velocity,
                    pitch=0
                )
            )
            nbs_instrument_set.add(pynbs_instrument_id)
        max_num_layers = len(notes_at_time) if len(notes_at_time) > max_num_layers else max_num_layers
        # Get the leftover midi notes of instrument i and then sort them into layers in the next iteration;
        # keep at it until we run out of midi notes
        instrument.notes = [note for note in instrument.notes if note not in notes_at_time]
    # Once done with a single instrument, add offset for what layer instrument (i+1)'s notes will
    # start to be inserted
    layer_offsets.append(layer_offsets[i] + max_num_layers)

# sort the nbs notes by tick in ascending order--required for saving
list_of_notes = sorted(list_of_notes, key=lambda note: note.tick)

# Testing for conflicting notes in same layer
for tick in range(max_num_ticks + 1):
    notes_at_tick = [note for note in list_of_notes if note.tick == tick]
    for i in range(len(notes_at_tick)):
        for j in range(len(notes_at_tick)):
            if j != i:
                note_i_layer = notes_at_tick[i].layer
                note_j_layer = notes_at_tick[j].layer
                if note_i_layer == note_j_layer:
                    print(str(notes_at_tick[i]), str(notes_at_tick[j], 'have the same layer and tick'))
                    exit(-1)

nbs_file = pynbs.new_file(
    song_name='song',
    tempo=20.00,
    song_layers=layer_offsets[-1],
    song_length=max_num_ticks
)
nbs_file.layers = [pynbs.Layer(id=i, name='', lock=False, volume=100, panning=0) for i in range(layer_offsets[-1])]
nbs_file.instruments = [pynbs.Instrument(id=i, name=nbs_instr_names[i], file=nbs_instr_files[i], pitch=45, press_key=True) for i in nbs_instrument_set]
nbs_file.notes.extend(list_of_notes)
# write to nbs_output.nbs in /content/song_files/nbs_output/combined_nbs.nbs
output_location = output_folder + "/pipeline4/pipeline4.nbs"
nbs_file.save(output_location, version=5)

In [ ]:
#@title Pipeline #5 MIDI to NBS Transcription
# get midi input from /content/song_files/combined_midi/combined_midi.mid
!pip install pretty_midi

pipeline5 =  output_folder + "/pipeline5/pipeline5_combined_midi.mid"
midi_file = PrettyMIDI(pipeline5)

# in many songs (and thus midi files), notes are played at the same time (same onset time)
# in nbs, layers are used when multiple notes are played at the same time
# so:
    # for each instrument
        # and for each note in that instrument
            # get the first note and its onset time
            # get all other notes with the same onset time
            # for each of those notes
                # create an nbs note and append it to the list of notes
                # each note should be put in a different layer (since they are played at the same time)
            # update the maximum number of layers
            # remove the notes we just converted from the midi instrument notes list
        # update layer offset (as to not override notes we just did for the previous instrument)

list_of_notes = []
layer_offsets = [0]
nbs_instrument_set = set()
max_num_ticks = 0
for i, instrument in enumerate(midi_file.instruments):
    max_num_layers = 0
    while(len(instrument.notes) > 0):
        curr_note = instrument.notes[0]
        time = curr_note.start
        # Filter midi notes according to nearly similar start time
        notes_at_time = list(filter(lambda n: n.start < time + 0.05, instrument.notes))
        # Convert all filtered notes to NBS notes and store all layers of instrument i's notes
        # in layer_offsets
        for j in range(len(notes_at_time)):
            max_num_ticks = int(round(time / 0.05)) if max_num_ticks < int(round(time / 0.05)) else max_num_ticks
            n = notes_at_time[j]
            pynbs_instrument_id = midi_to_nbs_instrs[instrument.program]
            list_of_notes.append(
                pynbs.Note(
                    tick=int(round(time / 0.05)),
                    layer=j + layer_offsets[i],
                    instrument=pynbs_instrument_id,
                    key=n.pitch - 21,
                    velocity=n.velocity,
                    pitch=0
                )
            )
            nbs_instrument_set.add(pynbs_instrument_id)
        max_num_layers = len(notes_at_time) if len(notes_at_time) > max_num_layers else max_num_layers
        # Get the leftover midi notes of instrument i and then sort them into layers in the next iteration;
        # keep at it until we run out of midi notes
        instrument.notes = [note for note in instrument.notes if note not in notes_at_time]
    # Once done with a single instrument, add offset for what layer instrument (i+1)'s notes will
    # start to be inserted
    layer_offsets.append(layer_offsets[i] + max_num_layers)

# sort the nbs notes by tick in ascending order--required for saving
list_of_notes = sorted(list_of_notes, key=lambda note: note.tick)

# Testing for conflicting notes in same layer
for tick in range(max_num_ticks + 1):
    notes_at_tick = [note for note in list_of_notes if note.tick == tick]
    for i in range(len(notes_at_tick)):
        for j in range(len(notes_at_tick)):
            if j != i:
                note_i_layer = notes_at_tick[i].layer
                note_j_layer = notes_at_tick[j].layer
                if note_i_layer == note_j_layer:
                    print(str(notes_at_tick[i]), str(notes_at_tick[j], 'have the same layer and tick'))
                    exit(-1)

nbs_file = pynbs.new_file(
    song_name='song',
    tempo=20.00,
    song_layers=layer_offsets[-1],
    song_length=max_num_ticks
)
nbs_file.layers = [pynbs.Layer(id=i, name='', lock=False, volume=100, panning=0) for i in range(layer_offsets[-1])]
nbs_file.instruments = [pynbs.Instrument(id=i, name=nbs_instr_names[i], file=nbs_instr_files[i], pitch=45, press_key=True) for i in nbs_instrument_set]
nbs_file.notes.extend(list_of_notes)
# write to nbs_output.nbs in /content/song_files/nbs_output/combined_nbs.nbs
output_location = output_folder + "/pipeline5/pipeline5.nbs"
nbs_file.save(output_location, version=5)

In [ ]:
#@title NBS to Audio Imports
!pip install nbswave

import nbswave
from pathlib import Path

In [ ]:
#@title Pipeline #1 NBS to Audio
# ffmpeg and ffprobe are required to use nbswave
# sound data from note block studio is also required

# get input from /content/song_files/nbs_output/combined_nbs.nbs

# Sounds data from Minecraft
song_name = input_file
sound_path = "/content/drive/MyDrive/beats-to-blocks/Sounds"
nbs_sounds_folder = Path(sound_path)

# write to /content/song_files/final_output/output.wav

nbs_location = output_folder + "/pipeline1/pipeline1.nbs"
audio_output_location = output_folder + "/pipeline1/p1_mc_" + song_name + ".wav"
nbswave.render_audio(
    nbs_location,
    audio_output_location,
    format='wav',
    default_sound_path=nbs_sounds_folder,
    ignore_missing_instruments=True
)

In [ ]:
#@title Pipeline #2 NBS to Audio
# ffmpeg and ffprobe are required to use nbswave
# sound data from note block studio is also required

# get input from /content/song_files/nbs_output/combined_nbs.nbs

# Sounds data from Minecraft
song_name = input_file
sound_path = "/content/drive/MyDrive/beats-to-blocks/Sounds"
nbs_sounds_folder = Path(sound_path)

# write to /content/song_files/final_output/output.wav

nbs_location = output_folder + "/pipeline2/pipeline2.nbs"
audio_output_location = output_folder + "/pipeline2/p2_mc_" + song_name + ".wav"
nbswave.render_audio(
    nbs_location,
    audio_output_location,
    format='wav',
    default_sound_path=nbs_sounds_folder,
    ignore_missing_instruments=True
)

In [ ]:
#@title Pipeline #3 NBS to Audio
# ffmpeg and ffprobe are required to use nbswave
# sound data from note block studio is also required

# get input from /content/song_files/nbs_output/combined_nbs.nbs

# Sounds data from Minecraft
song_name = input_file
sound_path = "/content/drive/MyDrive/beats-to-blocks/Sounds"
nbs_sounds_folder = Path(sound_path)

# write to /content/song_files/final_output/output.wav

nbs_location = output_folder + "/pipeline3/pipeline3.nbs"
audio_output_location = output_folder + "/pipeline3/p3_mc_" + song_name + ".wav"
nbswave.render_audio(
    nbs_location,
    audio_output_location,
    format='wav',
    default_sound_path=nbs_sounds_folder,
    ignore_missing_instruments=True
)

In [ ]:
#@title Pipeline #4 NBS to Audio
# ffmpeg and ffprobe are required to use nbswave
# sound data from note block studio is also required

# get input from /content/song_files/nbs_output/combined_nbs.nbs

# Sounds data from Minecraft
song_name = input_file
sound_path = "/content/drive/MyDrive/beats-to-blocks/Sounds"
nbs_sounds_folder = Path(sound_path)

# write to /content/song_files/final_output/output.wav

nbs_location = output_folder + "/pipeline4/pipeline4.nbs"
audio_output_location = output_folder + "/pipeline4/p4_mc_" + song_name + ".wav"
nbswave.render_audio(
    nbs_location,
    audio_output_location,
    format='wav',
    default_sound_path=nbs_sounds_folder,
    ignore_missing_instruments=True
)

In [ ]:
#@title Pipeline #5 NBS to Audio
# ffmpeg and ffprobe are required to use nbswave
# sound data from note block studio is also required

# get input from /content/song_files/nbs_output/combined_nbs.nbs

# Sounds data from Minecraft
song_name = input_file
sound_path = "/content/drive/MyDrive/beats-to-blocks/Sounds"
nbs_sounds_folder = Path(sound_path)

# write to /content/song_files/final_output/output.wav

nbs_location = output_folder + "/pipeline5/pipeline5.nbs"
audio_output_location = output_folder + "/pipeline5/p5_mc_" + song_name + ".wav"
nbswave.render_audio(
    nbs_location,
    audio_output_location,
    format='wav',
    default_sound_path=nbs_sounds_folder,
    ignore_missing_instruments=True
)